In [147]:
import pandas as pd
import numpy as np
import os
from bokeh.plotting import figure
from bokeh.io import output_file, show, save
from functies_voor_selectief_plotten import *
from sklearn.metrics import mean_squared_error

In [148]:
# controleer altijd of je in de map helpfiles zit
print(os.getcwd())

C:\Users\boble\Documents\GitHub\DataProcessing\datasets


In [149]:
from select_df import select_df

df_foodprices = select_df(1, 'foodprices')
df_migration = select_df(1, 'migration_movements')
df_weather = select_df(1, 'temperature_and_precipitation')

Succesfully imported foodprices!
Succesfully imported migration_movements!
succesfully imported temperature_and_precipitation!


In [87]:
# compare_refugees_Gambia_and_price_of_Rice (paddy, long grain, local)

def plot_scatter(country, product, year1, year2):
    x1 = normalize(select_plot_foodprices_average(df_foodprices, country, product, year1, year2), 'average_price')['Normalized_data']  
    y1 = normalize(select_plot_migration_movements(df_migration, country, year1, year2), 'Value')['Normalized_data']    
   
    output_file("Line.html")

    # Create a figure (that's 600x600)
    f = figure(plot_width=600, plot_height=600)
    
    f.xaxis.axis_label = "Normalised foodprices"
    f.yaxis.axis_label = "Normalised migration movements"

    # Plot the line
    f.circle(x1, y1, color='red')

    show(f)
    
plot_scatter('Gambia', 'Rice (paddy, long grain, local)', 2005, 2018)

c:\users\boble\appdata\local\programs\python\python36-32\lib\site-packages\bokeh\models\sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 122), ('y', 157)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


In [181]:
def plot_scatter_regression(country, product):
    # selecteer de input datasets
    dataset_x = normalize(select_plot_foodprices_average(df_foodprices, country, product, 1990, 2020), 'average_price')
    dataset_y = normalize(select_plot_migration_movements(df_migration, country, 1990, 2020), 'Value')   

    # de lijsten die worden gegenereerd
    covered_months = []
    output_x = []
    output_y = []
    
    # maak het ontwerp van de grafiek
    f = figure(plot_width=600, plot_height=600)
    f.title.text="Price of {} in {} over time".format(product, country)
    f.xaxis.axis_label = "Years"
    f.yaxis.axis_label = "Normalised migration movements"

    # vergelijk de mogelijke maanden en zet ze in een lijst wanneer ze gelijk zijn
    for month_x in dataset_x.year:
        for month_y in dataset_y.Timestamp:    
            if round(month_x, 4) == round(month_y, 4):
                output_x.append(float(dataset_x.loc[dataset_x['year'] == month_x]['Normalized_data']))
                output_y.append(float(dataset_y.loc[dataset_y['Timestamp'] == month_y][0:1]['Normalized_data']))
                covered_months.append(month_x)

    # regression line    
    regression_x = np.vstack(output_x)
    regression_x = np.column_stack((regression_x, np.ones(regression_x.shape[0])))
    regression_y = output_y
    
    x = pd.Series(output_x)
    # Now get out m and b values for our best fit line
    a, b = np.linalg.lstsq(regression_x, regression_y)[0]
    
    print(a, b)
    print(x)
    # plot the graph
    f.circle(output_x, output_y, color='red')
    f.line(x, a * x + b, color='blue')
    
    show(f)
    
plot_scatter_regression('Gambia', 'Rice (paddy, long grain, local)')


c:\users\boble\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


0.6942034783314577 -0.04091440372532081
0      0.013333
1      0.013333
2      0.013333
3      0.031851
4      0.031851
5      0.031851
6      0.031851
7      0.031851
8      0.031851
9      0.050368
10     0.050368
11     0.050368
12     0.050368
13     0.050368
14     0.050368
15     0.050368
16     0.050368
17     0.050368
18     0.050368
19     0.050368
20     0.000000
21     0.000000
22     0.022857
23     0.022857
24     0.057143
25     0.022857
26     0.022857
27     0.022857
28     0.053333
29     0.020000
         ...   
97     0.510000
98     0.444000
99     0.435000
100    0.419200
101    0.419200
102    0.565943
103    0.583086
104    0.564000
105    0.580000
106    0.600000
107    0.615000
108    0.890000
109    0.730000
110    0.776542
111    0.679481
112    0.605264
113    0.625264
114    0.765714
115    0.680000
116    0.346667
117    0.760000
118    0.400000
119    0.798422
120    0.910000
121    0.919474
122    0.870000
123    0.800000
124    0.800000
125    0.915000


In [83]:
def plot_regression(country, product, year1, year2):
    # Set up X as median room values and use Use vstack to make X two-dimensional
    # Remember that X normally is (N,) instead of (N,1).
    X = np.vstack(normalize(select_plot_foodprices_average(df_foodprices, country, product, year1, year2), 'average_price')['Normalized_data'])
    X = np.column_stack((X, np.ones(X.shape[0])))
    
    # Set up Y as the target price of the houses.
    Y = normalize(select_plot_migration_movements(df_migration, country, year1, year2), 'Value')['Normalized_data']
    
    x = normalize(select_plot_foodprices_average(df_foodprices, country, product, year1, year2), 'average_price')['Normalized_data']

    # Now get out m and b values for our best fit line
    a, b = np.linalg.lstsq(X, Y)[0]
    f.line(x, a * x + b, color='red')
    
    show(f)

plot_regression('Zimbabwe', 'Maize', 2010, 2014)   

LinAlgError: Incompatible dimensions

In [175]:
# compare_refugees_Gambia_and_price_of_Rice (paddy, long grain, local)

def plot_timeline(country, product, year1, year2):
    # scatter
    x1 = select_plot_foodprices_average(df_foodprices, country, product, year1, year2)['year']
    y1 = normalize(select_plot_foodprices_average(df_foodprices, country, product, year1, year2), 'average_price')['Normalized_data']    
    
    # regression line    
    x2 = np.vstack(select_plot_foodprices_average(df_foodprices, country, product, year1, year2)['year'])
    x2 = np.column_stack((x2, np.ones(x2.shape[0])))
    y2 = normalize(select_plot_foodprices_average(df_foodprices, country, product, year1, year2), 'average_price')['Normalized_data']
    x = select_plot_foodprices_average(df_foodprices, country, product, year1, year2)['year']
    # Now get out m and b values for our best fit line
    a, b = np.linalg.lstsq(x2, y2)[0]

    print(x)

    # set the layout
    f = figure(plot_width=600, plot_height=600)
    
    f.title.text="Price of {} in {} over time".format(product, country)
    f.xaxis.axis_label = "Years"
    f.yaxis.axis_label = "Normalised migration movements"

    # plot the graph
    f.circle(x1, y1, color='red')
    f.line(x, a * x + b, color='blue')

    show(f)
    
plot_timeline('Gambia', 'Rice (paddy, long grain, local)', 2005, 2018)

c:\users\boble\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


0      2006.000000
1      2006.083333
2      2006.166667
3      2006.250000
4      2006.333333
5      2006.416667
6      2006.500000
7      2006.583333
8      2006.666667
9      2006.750000
10     2006.833333
11     2006.916667
12     2007.083333
13     2007.166667
14     2007.250000
15     2007.333333
16     2007.416667
17     2007.500000
18     2007.583333
19     2007.666667
20     2007.750000
21     2007.833333
22     2007.916667
23     2008.000000
24     2008.083333
25     2008.166667
26     2008.250000
27     2008.333333
28     2008.416667
29     2008.500000
          ...     
92     2014.000000
93     2014.083333
94     2014.166667
95     2014.250000
96     2014.333333
97     2014.416667
98     2014.500000
99     2014.583333
100    2015.750000
101    2015.833333
102    2016.000000
103    2016.166667
104    2016.250000
105    2016.333333
106    2016.416667
107    2016.500000
108    2016.583333
109    2016.666667
110    2016.750000
111    2016.833333
112    2016.916667
113    2017.